In [2]:
!pip install numpy==1.19.5 --user

In [3]:
!pip install h5py==2.10.0 --user

In [4]:
!pip install gensim==3.6.0 --user

In [5]:
!pip install deepcut --user

In [6]:
!pip install pythainlp --user

In [7]:
!pip install nltk --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
!pip install pandas --user

In [14]:
!pip install flask --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
from pythainlp.corpus import thai_stopwords
import deepcut
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

import itertools
import datetime
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint

import difflib

In [13]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [15]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import json

In [16]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [17]:
import gensim
wv_model = gensim.models.Word2Vec.load('corpus.th.model')

In [18]:
def word2idx(word):
    index = 0
    index = wv_model.wv.vocab[word].index
    return index

In [19]:
def word_index(listword):
    dataset = []
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    for sentence in listword:
        tmp = []
        for w in sentence:
            if w not in wv_model:
                continue

            if w not in vocabulary:
                vocabulary[w] = len(inverse_vocabulary)
                tmp.append(len(inverse_vocabulary))
                inverse_vocabulary.append(w)
            else:
                tmp.append(word2idx(w))
        dataset.append(tmp)
    return np.array(dataset)

In [20]:
# define word embedding
vocab_list = [(k, wv_model.wv[k]) for k, v in wv_model.wv.vocab.items()]
embeddings_matrix = np.zeros((len(wv_model.wv.vocab.items()) + 1, wv_model.vector_size))
for i in range(len(vocab_list)):
    word = vocab_list[i][0]
    embeddings_matrix[i + 1] = vocab_list[i][1]

In [21]:
# vocab_list

In [22]:
EMBEDDING_DIM = 300
embeddings_matrix = 1 * np.random.randn(len(vocab_list) + 1, EMBEDDING_DIM)  # This will be the embedding matrix
embeddings_matrix[0] = 0  # So that the padding will be ignored

In [23]:
# Model variables
n_hidden = 256
batch_size = 128
n_epoch = 100
max_seq_length = 2704

In [24]:
embeddings_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.76014451, -0.73600445,  0.89420246, ...,  0.24733923,
         1.00154121, -0.05668638],
       [-0.62274693, -0.21273356, -0.19482881, ...,  1.27856355,
        -1.19812914,  1.41303061],
       ...,
       [-0.41098705,  0.88387773, -1.47247226, ...,  0.41173911,
         1.20978042, -1.20721838],
       [ 1.41097323,  0.90917485, -0.78205871, ...,  0.5097654 ,
        -0.31080312,  0.18946268],
       [ 1.47938183, -0.20076783, -0.19750797, ...,  0.28850249,
        -0.72778715,  0.21661161]])

In [25]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [26]:
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings_matrix), EMBEDDING_DIM, weights=[embeddings_matrix], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])


malstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Start training
training_start_time = time()

In [27]:
malstm.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2704)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2704, 300)    9468300     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 256)          570368      embedding[0][0]              

In [28]:
# Load best weight from model
malstm.load_weights('sm_colab_ka.h5')

#Test with Text

In [29]:
def prepare_for_predict(input_questions):
    q_input= []
    cleansing(input_questions)
    tokenized_input_1 =deepcut.tokenize(input_questions)
    for sentence in tokenized_input_1:
      q_input.append(sentence)
    q_input= word_index(tokenized_input_1)
    q_input = pad_sequences(q_input, maxlen=max_seq_length)
    return q_input

In [30]:
#Duplicate list
def duplicate(testList, n):
    return [ele for ele in testList for _ in range(n)]

In [31]:
from itertools import chain
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [32]:
#หลักสูตร
curriculum = ['วิชา CPE332/Professional issuesมีวิชาตัวต่อมั้ยครับ?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรปกติ?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?',
  'วิชา CPE375/Interactive computingมีวิชาตัวต่อมั้ยครับ?',
  'หน่วยกิตที่ต้องเก็บให้ครบก่อนจบหรือไม่?',
  'วิศวะคอม/วิศวกรรมคอมพิวเตอหลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE224/Computer architectures มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกที่มีวิชาบังคับลงก่อน หลักสูตรนานาชาติ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ มีกี่หน่วยกิต? ',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE212/Algorithm มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอมหลักสูตรวิทยาศาตร์ข้อมูลสุขภาพมีอัตราค่าเรียนเท่าไหร่',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 3 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลักสูตรปกติ?',
  'วิชา CPE314/Computer networksมีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 1-2 เรียนอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิชา CPE327/Software engineering มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอมพิวเตอร์แต่ละหลักสูตรมีระยะเวลาในการศึกษากี่ปี?',
  'วิชา CPE325/Big data มีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE111/Data Structure มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 2 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาเลือกเสรีต้องลงวิชานอกภาคเท่านั้นหรือไม่?',
  'วิศวะคอมหลักสูตรปกติมีอัตราค่าเรียนเท่าไหร่',
  'วิชา CPE326/Operating systems มีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE121/Discrete มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ มีกี่หน่วยกิต?',
  'วิชา CPE342/Java programmingมีวิชาตัวต่อมั้ยครับ?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรวิทยาศาสตรบัณฑิต\nสาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ ?',
  'วิชา CPE343/Object orientedมีวิชาตัวต่อมั้ยครับ?',
  'วิชา CPE100/Programming มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ มีกี่หน่วยกิต?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิชา CPE223/Digital มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรนานาชาติปี 4 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE101/Exploration มีวิชาตัวต่อมั้ยครับ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรเรสิเดนทอล คอเลจ ปีที่ 3-4 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรปกติ ?',
  'วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 3 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรวิทยาศาตร์ข้อมูลสุขภาพ ปีที่1-2 เรียนอะไรบ้าง?',
  'วิชา CPE231/Database มีวิชาตัวต่อมั้ยครับ?',
  'วิชาภาคบังคับที่มีวิชาต่อเนื่อง  หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nมีอะไรบ้าง ?',
  'วิชาภาษาอังกฤษ LNG มีเกณท์การเรื่มเรียนอย่างไร หลักสูตรนานาชาติ ?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 2 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอมหลักสูตรปกติมีระยะเวลาในการศึกษากี่ปี?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE213/Data model วิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 1 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?',
  'วิศวะคอมหลักสูตรนานาชาติมีอัตราค่าเรียนเท่าไหร่',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 4 เทอม 1 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิชา CPE122/Circuits มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรปกติ  ปี 4 เทอม 2\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรนานาชาติ ปีที่ 3 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรนานาชาติ ปี 3 เทอม 1\nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิชา CPE329/Business intelligence มีวิชาตัวต่อมั้ยครับ?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาศาสตร์ข้อมูลสุขภาพ \nปี 2 เทอม 2 ต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'วิศวกรรมคอมพิวเตอร์หลักสูตรปกติ ปีที่ 4 เรียนอะไรบ้าง?',
  'วิศวะคอม/หลักสูตรปกติ ปี 1 เทอม 1 \nต้องลงเรียนกี่หน่วยกิต? เรียนวิชาอะไรบ้าง?',
  'ชำระเงินค่าลงทะเบียนแล้ว แต่ในระบบยังขึ้นว่าไม่ชำระเงิน',
  'ถ้าต้องการลงทะเบียนเรียนน้อยกว่า 9 หน่วยกิตต้องทำอย่างไร?',
  'ลากิจต้องทำอย่างไร?',
  'ผมอยากลาออกต้องทำยังไง?',
  'ใบแจ้งชำระเงินค่าลงทะเบียน สั่งพิมพ์ได้จากที่ไหน',
  'แจ้งขอสำเร็จการศึกษาได้ที่ไหน',
  'ถ้าวิชาเรียน sec เต็มแล้วทำยังไง',
  'ถ้าจะจบการศึกษา แต่เรียน gen ซ้ำหมวดกันทำยังไง',
  'ลงทะเบียนเรียนล่าช้าใช้เอกสารอะไรบ้าง',
  'หากมีเหตุต้องพักการเรียน สามารถพักการเรียนตอนนี้ได้ไหม ถ้าได้ต้องทำอย่างไรบ้าง?',
  'ดรอปได้ถึงเมื่อไหร่?',
  'สามารถเช็คกำหนดการลงทะเบียนเรียนได้ที่ไหน?',
  'ต้องการสอบโดยไม่เข้าเรียนได้ไหม?',
  'ยังต้องเข้าไปเรียนที่มหาลัยไหม?',
  'ลืมจ่ายเงินค่าลงทะเบียน และ ใบแจ้งหนี้เกินกำหนดไปแล้ว ทำยังไง?',
  'ลาป่วยต้องทำอย่างไร? ',
  'ตารางสอบซ้อนต้องทำยังไง?',
  'ฉันสามารถลงวิชาเสริมในเทอมนี้ได้มากที่สุดกี่ตัว',
  'ถ้าจะต้องการเอาเครื่องคิดเลขเข้้าห้องสอบต้องทำยังไง',
  'ถ้าต้องการลงทะเบียนเรียนมากกว่า 19 หน่วยกิตต้องทำอย่างไร?',
  'กรณีเพิ่มและหลักสูตรปิดรายวิชา จะได้รับเงินคืนไหม หรือ ต้องจ่ายเงินเพิ่มเท่าไหร่',
  'ลงทะเบียนเรียนเพิ่มรายวิชา ชำระเงินได้ที่ไหน?',
  'อยากได้เอกสารเบิกค่าเทอมต้องเตรียมอะไรบ้าง',
  'ตอนดรอปต้องขอลายเซ็นใครบ้าง?',
  'ตรวจวันเวลาสอบได้ที่ไหน',
  'สามารถลงทะเบียนเกินหน่วยกิตที่หลักสูตรกำหนดได้หรือไม่',
  'เรียนต่อต่างประเทศต้องใช้เอกสารอะไรบ้าง',
  'หากจบการศึกษาไม่ทันเกณฑ์ทหารทำยังไง',
  'ลงทะเบียนล่าช้าหลังระบบลงทะเบียนปิดที่ไหน',
  'ถ้าการย้ายหลักสูตรต้องทำอย่างไร?',
  'กรณีเพิ่มและลดรายวิชา จะได้รับเงินคืนไหม หรือ ต้องจ่ายเงินเพิ่มเท่าไหร่',
  'สามารถเช็คคอร์สเรียนออนไลน์ได้ที่ไหน',
  'ถ้าจะดึงบางวิชาลงมาเรียนแทนตัวที่ติด f ได้ไหม ต้องทำอย่างไร',
  'วิชาเลือกไหนน่าสนใจ?',
  'ขอทรานสคริปต์ออนไลน์ได้ที่ไหน',
  'ถ้าต้องการเปลี่ยน section ต้องทำอย่างไร',
  'ขอลดรายวิชาออนไลน์ หลังหมดเขต เพิ่ม-ลด 2 สัปดาห์แรกได้ที่ไหน?',
  'วิศวะคอม/ TCAS รอบ 2 มีโครงการไหนเปิดรับบ้าง?',
  'ถ้าจะสมัครเข้าโครงการ Active Recruitment วัดผลยังไง? ดูอะไรบ้าง?',
  'วิศวะคอม/ TCAS รอบ 4 ใช้คะแนนสอบอะไรบ้าง?',
  'หลักสูตรปกติมีเกณฑ์การรับยังไงบ้าง?',
  'หลักสูตรHDSมีเกณฑ์การรับยังไงบ้าง?',
  'สมัครเข้าโครงการ Active Recruitment ได้อย่างไร?',
  'วิศวะคอม/ TCAS รอบ 3ใช้คะแนนสอบอะไรบ้าง?',
  'หลักสูตรนานาชาติมีเกณฑ์การรับยังไงบ้าง?',
  'ถ้าสนใจจะเรียนวิศวะคอม/ เปิดรับเมื่อไหร่ ช่วงไหนบ้าง?',
  'วิศวะคอม/ สามารถสมัครเข้าเรียนยังไงได้บ้าง มีรอบอะไรเปิดรับบ้าง?',
  'แต่ละหลักสูตรมีเกณฑ์การรับพื้นฐานยังไงบ้าง?',
  'ถ้าจะสมัครเข้าโครงการ Active Recruitment ใช้เอกสารอะไรบ้าง?',
  'วิศวะคอม/ TCAS รอบ 1 มีโครงการไหนเปิดรับบ้าง?',
  'การับสมัครเด็กเข้าเรียน มีทั้งหมดกี่รอบ? อะไรบ้าง?',
  'สามารถติดต่อข่าวสารช่องทางการรับสมัครได้ที่ช่องทางไหนบ้าง? ',
  'จะต้องใช้เอกสารอะไรบ้างในการสมัครเข้าศึกษา',
  'ถ้าไม่ได้เป็นนักศึกษาชั้นปีที่ 3 สามารถฝึกงานได้หรือไม่？',
  'หากไม่สามารถหาที่ฝึกงานในช่วงนี้ได้ จะต้องดรอปวิชาฝึกงานไหม',
  'Work Integrate Leaning หรือ Wil จำเป็นที่ลงเลือกวิชาเลือกเสรีหรือไม่?',
  'หากไม่่สามารถฝึกงานได้ต้องทำอย่างไร',
  'ถ้าต้องการฝึกงานต้องทำอย่างไรบ้าง?',
  'ฝึกงานจำเป็นต้องครบ40ชั่วโมงไหม？',
  'Work Integrate Leaning หรือ Wil ทำตั้งแต่ช่วงเวลาไหนถึงไหน?',
  'ถ้าต้องการทำ Work Integrate Leaning หรือ Wil ต้องทำอย่างไร?',
  'เวลาต้องการยื่นเอกสารต้องยื่นที่ไหน',
  'วิศวะคอม/หลักสูตรเรสิเดนทอล คอลเลจจบแล้วได้ปริญญาอะไร?',
  'ขั้นตอนและช่วงเวลาเปิดรับสมัครนักศึกษา?',
  'หลักสูตรวิทยาศาสตร์ข้อมูลสุขภาพจบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/หลักสูตรปกติจบแล้วได้ปริญญาอะไร?',
  'เรียนจบต้องใช้คะแนนอังกฤษอะไรบ้าง ขั้นตํ่าเท่าไหร่??',
  'หลักสูตรนานาชาติจบมาแล้วทำงานในสายใดบ้าง?',
  'สามารถติดต่ออ.ได้ช่องทางไหนบ้าง?',
  'สมัครหลักสูตรยังไง?',
  'ขั้นตอนการรับสมัครโครงการทุนการศึกษา?',
  'วิศวะคอม/เรียนเกี่ยวกับอะไร?',
  '/วิศวะคอมพิวเตอร์ เรียนจบทำงานอะไร',
  'การส่งคะแนนสอบภาษาอังกฤษเพื่อสำเร็จการศึกษา ส่งอย่างไร?',
  'ใครเป็นหัวหน้าภาควิชาวิศวะคอม/?',
  'มีที่ฝึกงานแนะนำไหม?',
  'ติดต่อวิศวะคอม/ได้ทางช่องทางไหนได้บ้าง?',
  'สามารถติดต่อได้ทางช่องทางไหนบ้าง ช่วงเวลากี่โมงถึงกี่โมง?',
  'หลักสูตรเรสิเดรเทียล คอเจน์จบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/แตกต่างจาก IT อย่างไร?',
  'วิศวะคอม/หลักสูตรวิทยาศาสตร์ข้อมูลสุขภาพจบแล้วได้ปริญญาอะไร?',
  'เกียรตินิยมอันดับ 2 อยู่ที่เท่าไหร่?',
  'หลักสูตรวิิทยาศาสตร์ข้อมูลสุขภาพคืออะไร?',
  'วิศวะคอม/จบไปทำงานอะไร?',
  'อาจารย์ในภาควิชาวิศวะคอม/มีใครบ้าง? ',
  'เกียรตินิยมอันดับ 1 อยู่ที่เท่าไหร่?',
  'หลักสูตรปกติจบมาแล้วทำงานในสายใดบ้าง?',
  'วิศวะคอม/แล้วได้ปริญญาอะไร?',
  'วิศวะคอม/หลักสูตรนานาชาติจบแล้วได้ปริญญาอะไร?',
  'วิศวะคอม/อยู่ที่ไหน?',
  'สามารถขอทุนอะไรบ้าง?',
  'ทุนเพชรพระจอมเกล้าภาควิชารับกี่คน？',
  'ทุนของภาควิชามีอะไรบ้าง？',
  'การสมัครกู้ยืมเรียน ทำอย่างไร?',
  'ถ้าอยากไปเรียนต่อต่างและขอทุนต่างประเมศต้องทำอย่างไรบ้าง？',
  'มีเงินไม่พอจ่าย ทำอะไรได้บ้าง?',
  'ทุนจ้างงานคืออะไร？',
  'ปัจจุบันมีทุนอะไรเปิดรับบ้าง?']

# time total

In [33]:
%%time
curriculumObj = {'curriculum':curriculum}
curriculumDF = pd.DataFrame(data=curriculumObj)
beforeTok = {}
beforeTok['หลักสูตร'] = curriculumDF
tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)
max_word = 19219
max_seq_length = 2704
q_category= []
for sentence in tokenized_curriculum:
    q_category.append(sentence)
q_category = word_index(q_category)
all_Question_categorylen = len(q_category)
all_Question_categorylen
input2 = 'วิศวคอมมีหลักสูตรอะไรบ้าง'
tokenized_input_2=cleansing(input2)
tokenized_input_2 =deepcut.tokenize(tokenized_input_2)
tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)
q_user = word_index(tokenized_dup_input_2)
# Split to dicts
M_input = {'left': q_category, 'right': q_user}
# Zero padding
for model_input, side in itertools.product([M_input], ['left', 'right']):
    model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

# Make sure everything is ok
assert M_input['left'].shape == M_input['right'].shape
play_predict = malstm.predict(x=[M_input['left'],  M_input['right']])
max_question_percentage = max(play_predict)
question_index = np.where(play_predict == max_question_percentage)
predictedQuestion = curriculum[question_index[0][0]]
print('output: '+ predictedQuestion+ ' ' + "%lf" % max_question_percentage)
value = {
  "predictedQuestion": predictedQuestion,
  "similarity": "%lf" % max_question_percentage
}

<ipython-input-19-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:
<ipython-input-19-e36182fd4f37>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dataset)


output: ติดต่อวิศวะคอม/ได้ทางช่องทางไหนได้บ้าง? 0.998395
Wall time: 46.9 s


# time seperate

In [34]:
%%time
curriculumObj = {'curriculum':curriculum}

Wall time: 0 ns


In [35]:
%%time
curriculumDF = pd.DataFrame(data=curriculumObj)

Wall time: 998 µs


In [36]:
%%time
beforeTok = {}
beforeTok['หลักสูตร'] = curriculumDF

Wall time: 0 ns


In [37]:
%%time
tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)

Wall time: 8.76 s


In [38]:
%%time
max_word = 19219
max_seq_length = 2704

Wall time: 0 ns


In [39]:
%%time
q_category= []
for sentence in tokenized_curriculum:
    q_category.append(sentence)

Wall time: 998 µs


In [40]:
%%time
q_category = word_index(q_category)

Wall time: 7.98 ms


<ipython-input-19-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:
<ipython-input-19-e36182fd4f37>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(dataset)


In [41]:
%%time
all_Question_categorylen = len(q_category)
all_Question_categorylen

Wall time: 0 ns


172

#Question from User

In [42]:
input2 = 'วิศวคอมมีหลักสูตรอะไรบ้าง'

In [43]:
%%time
tokenized_input_2=cleansing(input2)

Wall time: 0 ns


In [44]:
%%time
tokenized_input_2 =deepcut.tokenize(input2)

Wall time: 72.8 ms


In [45]:
%%time
tokenized_dup_input_2= duplicate([tokenized_input_2],all_Question_categorylen)

Wall time: 0 ns


In [46]:
%%time
q_user = word_index(tokenized_dup_input_2)

Wall time: 2.99 ms


<ipython-input-19-e36182fd4f37>:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w not in wv_model:


In [47]:
%%time
# Split to dicts
M_input = {'left': q_category, 'right': q_user}

Wall time: 0 ns


In [48]:
%%time
# Zero padding
for model_input, side in itertools.product([M_input], ['left', 'right']):
    model_input[side] = pad_sequences(model_input[side], maxlen=max_seq_length)

Wall time: 3.99 ms


In [49]:
%%time
# Make sure everything is ok
assert M_input['left'].shape == M_input['right'].shape

Wall time: 0 ns


In [50]:
%%time
play_predict = malstm.predict(x=[M_input['left'],  M_input['right']])

Wall time: 37.7 s


In [51]:
%%time
max_question_percentage = max(play_predict)

Wall time: 0 ns


In [52]:
%%time
question_index = np.where(play_predict == max_question_percentage)

Wall time: 0 ns


In [53]:
%%time
predictedQuestion = beforeTok['หลักสูตร'].loc[question_index[0][0]]

Wall time: 0 ns


In [54]:
%%time
value = {
          "predictedQuestion": str(predictedQuestion),
          "similarity": "%lf" % max_question_percentage
        }

Wall time: 998 µs


In [55]:
%%time
json.dumps(value, ensure_ascii=False).encode('utf8')

Wall time: 0 ns


b'{"predictedQuestion": "curriculum    \xe0\xb8\x95\xe0\xb8\xb4\xe0\xb8\x94\xe0\xb8\x95\xe0\xb9\x88\xe0\xb8\xad\xe0\xb8\xa7\xe0\xb8\xb4\xe0\xb8\xa8\xe0\xb8\xa7\xe0\xb8\xb0\xe0\xb8\x84\xe0\xb8\xad\xe0\xb8\xa1/\xe0\xb9\x84\xe0\xb8\x94\xe0\xb9\x89\xe0\xb8\x97\xe0\xb8\xb2\xe0\xb8\x87\xe0\xb8\x8a\xe0\xb9\x88\xe0\xb8\xad\xe0\xb8\x87\xe0\xb8\x97\xe0\xb8\xb2\xe0\xb8\x87\xe0\xb9\x84\xe0\xb8\xab\xe0\xb8\x99\xe0\xb9\x84\xe0\xb8\x94\xe0\xb9\x89\xe0\xb8\x9a\xe0\xb9\x89\xe0\xb8\xb2\xe0\xb8\x87?\\nName: 150, dtype: object", "similarity": "0.998395"}'